## MNIST CNN

In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets('../data/MNIST_data/', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../data/MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../data/MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ../data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../data/MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
type(mnist)

tensorflow.contrib.learn.python.learn.datasets.base.Datasets

### Helper function

In [5]:
# Init Weights
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape=shape, stddev=0.1)
    return tf.Variable(init_random_dist)

In [6]:
# Init Bias
def init_bias(shape):
    init_bias_vals = tf.constant(0.1, shape=shape)
    return tf.Variable(init_bias_vals)

In [7]:
# CONV2D
def conv2d(x, W):
    # x --> [batch, H, W, Channels]
    # W --> [filter H, filter W, Channels IN, Channel Out]
    
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

In [8]:
# Pooling
def max_pool_2by2(x):
    # x --> [batch, H, W, Channels]
    # concern H, W
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1],padding='SAME')

In [9]:
# Conv Layer
def convolutional_layer(input_x, shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x, W)+b)

In [10]:
# Fully connected
def fully_connected_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer, W) + b

In [11]:
# Placeholder
x = tf.placeholder(tf.float32, shape=[None, 784])

In [12]:
y_true = tf.placeholder(tf.float32,shape=[None, 10])

In [13]:
# Layers
x_image = tf.reshape(x, [-1, 28, 28, 1])

In [14]:
# shape = [H,W,C,feature]
convo_1 = convolutional_layer(x_image, shape=[5,5,1,32])
convo_1_pooling = max_pool_2by2(convo_1)

Instructions for updating:
Colocations handled automatically by placer.


In [15]:
conv_2 = convolutional_layer(convo_1_pooling, shape=[5,5,32,64])
conv_2_pooling = max_pool_2by2(conv_2)

In [16]:
conv_2_flat = tf.reshape(conv_2_pooling, [-1, 7*7*64])
fully_layer_one = tf.nn.relu(fully_connected_layer(conv_2_flat, 1024))

In [17]:
# Dropout
hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(fully_layer_one, keep_prob=hold_prob)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [18]:
y_predict = fully_connected_layer(full_one_dropout, 10)

In [19]:
# Loss function
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_predict))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [20]:
# Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(cross_entropy)

In [21]:
init = tf.global_variables_initializer()

In [23]:
steps = 1000
with tf.Session() as sess:
    sess.run(init)
    for i in range(steps):
        batch_x, batch_y = mnist.train.next_batch(50)
        
        sess.run(train, feed_dict={x:batch_x, y_true:batch_y, hold_prob:0.5})
        
        if i%100 == 0:
            print("On step:{}".format(i))
            print("Accuracy: ")
            matched = tf.equal(tf.argmax(y_predict, 1), tf.argmax(y_true, 1))
            acc = tf.reduce_mean(tf.cast(matched, tf.float32))
            print(sess.run(acc, feed_dict={x:mnist.test.images, y_true:mnist.test.labels, hold_prob:1.0}))
            print("\n")
            

On step:0
Accuracy: 
0.0974


On step:100
Accuracy: 
0.9433


On step:200
Accuracy: 
0.9606


On step:300
Accuracy: 
0.9685


On step:400
Accuracy: 
0.9736


On step:500
Accuracy: 
0.9787


On step:600
Accuracy: 
0.9777


On step:700
Accuracy: 
0.9817


On step:800
Accuracy: 
0.9832


On step:900
Accuracy: 
0.983


